In [1]:
using YFinance, DataFrames, CSV

In [2]:
startdt = "2022-08-16"
enddt = "2023-10-26";

In [3]:
cac40 = get_prices.("^FCHI",startdt=startdt, enddt=enddt);

In [18]:
datas = cac40["timestamp"]
adjclose = cac40["adjclose"]
df = DataFrame()
df[:,"Data"] = datas
df[:,"CAC40"] = adjclose;
df[1:5,:]

Row,Data,CAC40
,DateTime,Float64
1,2022-08-16T07:00:00,6592.58
2,2022-08-17T07:00:00,6528.32
3,2022-08-18T07:00:00,6557.4
4,2022-08-19T07:00:00,6495.83
5,2022-08-22T07:00:00,6378.74


In [13]:
#"URW.PA","MT" ficaram de fora por não ter preços neste período
tickers = ["MC.PA","OR.PA","RMS.PA","TTE.PA","SAN.PA","AIR.PA","SU.PA","AI.PA","EL.PA","BNP.PA","CS.PA","SAF.PA","DG.PA","STLAP.PA","KER.PA","TEP.PA","WLN.PA","VIV.PA","ALO.PA","ERF.PA","RNO.PA","CA.PA","EN.PA","PUB.PA","GLE.PA","VIE.PA","ML.PA","LR.PA","CAP.PA","SGO.PA","ORA.PA","HO.PA","ACA.PA","STMPA.PA","ENGI.PA","BN.PA","RI.PA","DSY.PA"];

In [6]:
dados = get_prices.(tickers,startdt=startdt, enddt=enddt);

In [19]:
for dado in dados
    # df[:,dado["ticker"]] = dado["adjclose"]
    datas = dado["timestamp"]
    adjclose = dado["adjclose"]
    df2 = DataFrame()
    df2[:,"Data"] = datas
    df2[:,dado["ticker"]] = adjclose
    df = leftjoin(df, df2, on=:Data)
end

In [22]:
println(size(df))
# Dropar dias com NaN
df = df[all.(!isnan, eachrow(df[:,2:end])), :]

println(size(df))

(308, 40)
(306, 40)


In [24]:
println(size(df))
df[1:5,:]

(306, 40)


Row,Data,CAC40,MC.PA,OR.PA,RMS.PA,TTE.PA,SAN.PA,AIR.PA,SU.PA,AI.PA,EL.PA,BNP.PA,CS.PA,SAF.PA,DG.PA,STLAP.PA,KER.PA,TEP.PA,WLN.PA,VIV.PA,ALO.PA,ERF.PA,RNO.PA,CA.PA,EN.PA,PUB.PA,GLE.PA,VIE.PA,ML.PA,LR.PA,CAP.PA,SGO.PA,ORA.PA,HO.PA,ACA.PA,STMPA.PA,ENGI.PA,BN.PA,RI.PA,DSY.PA
,DateTime,Float64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,2022-08-16T07:00:00,6592.58,688.773,357.716,1380.18,47.4635,83.4747,108.988,133.734,133.765,159.638,46.8634,23.1368,111.369,91.6649,13.65,550.425,304.679,44.02,9.09252,23.8081,75.0489,30.5094,16.7366,29.0951,49.7946,21.935,24.1728,25.9956,80.98,184.365,44.429,9.61427,121.856,9.15832,37.2889,11.797,52.3497,189.513,42.5295
2,2022-08-17T07:00:00,6528.32,690.252,359.442,1384.15,47.5003,78.7317,106.622,132.773,133.058,159.442,46.2511,23.0098,110.537,90.6636,13.4705,549.84,302.225,43.83,8.93465,22.1832,74.6557,29.7503,16.7366,28.7361,48.9109,21.6194,23.7118,25.0298,80.04,182.603,43.055,9.64602,123.03,9.02699,35.6711,11.7733,52.3013,190.148,42.2858
3,2022-08-18T07:00:00,6557.4,696.657,358.752,1398.05,48.6616,78.7799,107.135,134.106,133.962,159.688,45.8351,22.9392,110.498,90.7984,13.6262,549.059,307.133,43.4,8.79431,22.0941,74.6361,30.0083,16.814,28.9061,48.8533,21.5266,23.3853,25.0872,80.56,183.435,41.9414,9.64228,121.953,8.92485,36.8224,11.7479,51.9049,190.197,42.6141
4,2022-08-19T07:00:00,6495.83,688.182,357.125,1387.13,48.5602,78.7221,104.749,131.832,134.04,158.852,44.5543,22.831,108.854,90.2978,13.225,544.571,304.385,42.77,8.75338,21.6977,73.5943,29.0854,16.8431,28.8022,48.8725,20.9093,23.174,24.6664,79.28,182.358,41.6811,9.69084,122.394,8.7388,36.346,11.7624,51.8759,190.831,42.3106
5,2022-08-22T07:00:00,6378.74,672.709,356.878,1368.75,48.8828,79.9946,101.791,126.029,129.661,153.744,43.4512,22.3792,106.794,88.8344,12.6938,527.398,299.183,42.01,8.75923,21.133,71.2158,27.8003,16.6931,28.3297,47.0859,20.2781,22.3481,23.8679,76.44,177.118,40.2493,9.66843,122.492,8.50533,35.3039,11.7406,51.4697,189.708,40.316


In [25]:
CSV.write("precos_projeto.csv",df)

"precos_projeto.csv"

In [26]:
function get_percent_return(prices::Vector{F}) where{F}
    return [(prices[i] - prices[i-1])/prices[i-1] for i in 2:length(prices)]
end

get_percent_return (generic function with 1 method)

In [27]:
retornos = DataFrame()
retornos[:,"Data"] = df.Data[2:end]
for (i,col) in enumerate(eachcol(df[:,2:end]))
    retornos[:,names(df)[i+1]] = get_percent_return(col)
end

In [34]:
CSV.write("retornos_projeto.csv",retornos)

"retornos_projeto.csv"